# Suspicious PowerShell Invocation based on Parent Process
Detects suspicious powershell invocations from interpreters or unusual programs

## Rule Content
```
- title: Suspicious PowerShell Invocation based on Parent Process
  id: 95eadcb2-92e4-4ed1-9031-92547773a6db
  status: experimental
  description: Detects suspicious powershell invocations from interpreters or unusual
    programs
  author: Florian Roth
  references:
  - https://www.carbonblack.com/2017/03/15/attackers-leverage-excel-powershell-dns-latest-non-malware-attack/
  tags:
  - attack.execution
  - attack.t1086
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      ParentImage:
      - '*\wscript.exe'
      - '*\cscript.exe'
      Image:
      - '*\powershell.exe'
    falsepositive:
      CurrentDirectory: '*\Health Service State\\*'
    condition: selection and not falsepositive
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Microsoft Operations Manager (MOM)
  - Other scripts
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_parent_path.keyword:(*\\wscript.exe OR *\\cscript.exe) AND process_path.keyword:(*\\powershell.exe)) AND (NOT (process_current_directory.keyword:*\\Health\ Service\ State\\*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()